In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.utils import resample
from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import os


from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler


import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score


import numpy as np
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score

from sklearn.ensemble import VotingClassifier, StackingClassifier


: 

# Import dataset

In [ ]:

df = pd.read_csv('/Users/victoriayuzova/Data-Science-Projects/Fraud Detection/creditcard.csv')
df.head()

# Explore dataset

Checking some basic things:
- if there are nulls,
- if there any interesting correlations between variables

In [ ]:
# all the variables are numeric, no need to worry about preprocessing categorical vars
# no need to worry about nulls (some algoritms dont tolerate them)
# good number of records
# time and amount variables are the only variables not encrypted

df.info()

In [ ]:
# dont see much here, but since these are numeric, let´s plot some correlations and cehck distributions of variables

df.describe().round(2)

In [ ]:
df.hist(figsize=(12, 10))

# time is bimodal
# here I see that some variables are skewed or have outliers; anomalies are likely signals of fraud

# not sure though what I can do about any of those distributions; further I will be tranforming them with StandardScaler

In [ ]:
# the dataset is imbalanced

df['Class'].value_counts(normalize = True)

In [ ]:
df['time_bin'] = pd.cut(df['Time'], bins=50)


df.groupby('time_bin')['Class'].mean().reset_index().plot(
    x='time_bin', y='Class', kind='line', figsize=(14, 4), alpha=0.5, title='Fraud Rate (%) Over Time'
)

df[df['Class'] == 1].groupby('time_bin')['Class'].count().reset_index().plot(
    x='time_bin', y='Class', kind='line', figsize=(14, 4), alpha=0.5, title='Fraud Rate (count) Over Time'
)

In [ ]:
# let´s check if there are any features that are highly correlated
# looks like there are no particularly strong correlations between variables
df.drop('time_bin', axis=1, inplace=True)

corr_matrix = df.corr().round(2)

plt.figure(figsize = (10,5))
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
df_downasampled = pd.concat([df[df['Class'] == 0].sample(492,random_state=42), df[df['Class'] == 1]])
df_downasampled['Class'].value_counts()

In [ ]:
# having downsampled the dataset, we can better see the correlation between features because fraud cases are more visible
# we will only use downsampling for analysis, not for modeling

corr_matrix = df_downasampled.corr().round(2)

plt.figure(figsize = (20,20))
sns.heatmap(corr_matrix, annot=True)
plt.show()

# Conclusions:
## Features with highest positive corr to fraud are 2, 4, 11, 19
## Features with highest negative corr to fraud are 3, 9, 10, 12, 14, 17, 18, 20

Correlation might not be the best when we have a binary variable and continuous feature. Let´s try doing t-test


In [ ]:
not_fraud = df_downasampled[df_downasampled['Class'] == 0]
fraud = df_downasampled[df_downasampled['Class'] == 1]

train_cols = [col for col in df_downasampled.columns if col != 'Class'] # choosing all continuous columns except target

# Run t-test for each feature

p_values = []

for c in train_cols:
    p = ttest_ind(not_fraud[c], fraud[c]).pvalue
    p_values.append({'variable': c, 'p_value': p})

p_values_df = pd.DataFrame(p_values)
p_values_df

In [ ]:
# ok, here I treid to exclude the variables that we not relevant but seems like every variable is valuable here

relevant_p_values = p_values_df[p_values_df['p_value']< 0.05]

relevant_p_values

# Modeling

First lets try a simple model (Logistic Regression) and let it handle the imbalanced class

In [ ]:
# split target and features

# sorting df by time to later apply TimeSeries CV
df = df.sort_values(by='Time')


X = df.drop(['Time','Class'], axis=1)  # features
y = df["Class"]               # target

# split into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.25, random_state=42)

#check sample size

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

In [ ]:
# With StandardScaler we need to bring different numerical variables to comparable scales before ingesting it into the model

std = StandardScaler()
std = std.fit(X_train)

X_train = std.transform(X_train)
X_test = std.transform(X_test)

Logistic Regression

In [ ]:
# here we´ll initialize the Time Serires Cross Validation

tscv = TimeSeriesSplit(n_splits=10)
filtered_splits = []

for i, (train_idx, val_idx) in enumerate(tscv.split(X_train), start=1):
    y_val = y_train.iloc[val_idx]
    positives = (y_val == 1).sum()
    
    if positives > 0:
        filtered_splits.append((train_idx, val_idx))
        print(f"✅ Fold {i} kept: Positives = {positives}")
    else:
        print(f"❌ Fold {i} skipped: Positives = {positives}")


Logistic Regression with Grid search

In [ ]:

pipe = Pipeline([
    ('ros', RandomOverSampler(random_state=42)),
    ('clf', LogisticRegression(max_iter=1000))
])

param_grid = {'clf__C':[0.001,0.01,0.1],'clf__class_weight':[None,'balanced']}
search_lr = GridSearchCV(pipe, param_grid, cv=filtered_splits, scoring='average_precision', n_jobs=-1)
search_lr.fit(X_train, y_train)

# Best parameters and best score
print("Best PR AUC (mean CV):", search_lr.best_score_)
print("Best parameters:", search_lr.best_params_)

# Average precision on the full training set with the best model
y_proba = search_lr.best_estimator_.predict_proba(X_train)[:, 1]
pr_auc_full = average_precision_score(y_train, y_proba)
print("PR AUC on full training set:", pr_auc_full)


In [ ]:

df = pd.read_csv('/Users/victoriayuzova/Data-Science-Projects/Fraud Detection/creditcard.csv')
df.head()

In [ ]:

df = pd.read_csv('/Users/victoriayuzova/Data-Science-Projects/Fraud Detection/creditcard.csv')
df.head()

In [ ]:

df = pd.read_csv('/Users/victoriayuzova/Data-Science-Projects/Fraud Detection/creditcard.csv')
df.head()

Random Forest with Grid Search

In [ ]:
pipe = Pipeline([
    ('ros', RandomOverSampler(random_state=42)),
    ('clf', RandomForestClassifier(random_state=42))
])

param_grid = {
    'clf__n_estimators': [100, 300],
    'clf__max_depth': [5, 7],
    'clf__max_features': ['sqrt'],
    'clf__criterion': ['gini']
}

search_rf = GridSearchCV(
    pipe, param_grid, cv=filtered_splits,
    scoring='average_precision', n_jobs=-1
)
search_rf.fit(X_train, y_train)

print("Best PR AUC (mean CV):", search_rf.best_score_)
print("Best parameters:", search_rf.best_params_)

y_proba = search_rf.best_estimator_.predict_proba(X_train)[:, 1]
pr_auc_full = average_precision_score(y_train, y_proba)
print("PR AUC on full training set:", pr_auc_full)


Gradient Boosting with Grid Search

In [ ]:
pipe = Pipeline([
    ('ros', RandomOverSampler(random_state=42)),
    ('clf', GradientBoostingClassifier(random_state=42))
])

param_grid = {
    'clf__learning_rate': [0.1],                
    'clf__n_estimators': [100],                 
    'clf__max_depth': [5],                      
    
}

search_gb = GridSearchCV(
    pipe, param_grid, cv=filtered_splits,
    scoring='average_precision', n_jobs=-1, return_train_score=False
)
search_gb.fit(X_train, y_train)

print("Best PR AUC (mean CV):", search_gb.best_score_)
print("Best parameters:", search_gb.best_params_)

y_proba = search_gb.best_estimator_.predict_proba(X_train)[:, 1]
print("PR AUC on full training set:", average_precision_score(y_train, y_proba))


# Compare the models

In [ ]:
X_test

In [ ]:
# Lets visualize precision recall curve for all three models

plt.figure(figsize=(10,10))

# Logistic Regression
prec, rec, _ = precision_recall_curve(y_test, search_lr.predict_proba(X_test)[:, 1])
ap_lr_test = average_precision_score(y_test, search_lr.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Random Forest
prec, rec, _ = precision_recall_curve(y_test, search_rf.predict_proba(X_test)[:, 1])
ap_rf_test = average_precision_score(y_test, search_rf.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Gradient Boosting
prec, rec, _ = precision_recall_curve(y_test, search_gb.predict_proba(X_test)[:, 1])
ap_gb_test = average_precision_score(y_test, search_gb.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

plt.xlabel("Recall - Of all the frauds in reality, how many did you catch?")
plt.ylabel("Precision - Of all the users you predicted as frauds, how many actually were frauds?")
plt.legend(["LR", "RF", "GB"])

# Print train/test AP
print("LR AP: Train {:.3f} || Test {:.3f}".format(search_lr.best_score_, ap_lr_test))
print("RF AP: Train {:.3f} || Test {:.3f}".format(search_rf.best_score_, ap_rf_test))
print("GB AP: Train {:.3f} || Test {:.3f}".format(search_gb.best_score_, ap_gb_test))
plt.hlines(y=0.001727, xmin=0, xmax=1, colors='red', linestyles='dashed', label='Baseline (random)')


# looks like the test is pretty different from what we´ve seen in train (not very good?)
# so here GB shows to be the best model; we can play with the threasholds and find optimal balance between precision and recall:


In [ ]:
# Here I want to see if I settle for catching 90% of all frauds, how many legitimate users will be wrongly flagged
# I think the final decision could depend on company's budget for false positives vs missed frauds

# Set your target recall (e.g., catch 90% of all frauds)
target_recall = 0.8

# Get predicted probabilities
y_probs = search_gb.predict_proba(X_test)[:, 1]

# Get precision-recall-thresholds
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)

# Find the first index where recall >= target (excluding the last point)
valid_idx = np.where(recalls[:-1] >= target_recall)[0]

if len(valid_idx) > 0:
    idx = valid_idx[-1]  # Take the last one to get highest precision at target recall
    threshold = thresholds[idx]
    
    y_pred = (y_probs >= threshold).astype(int)

    cm_gb = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_gb)
    disp.plot(cmap='Blues')

    print(f"Threshold for ≥ {target_recall:.2f} recall: {threshold:.3f}")
    print(f"Recall:    {recalls[idx]:.3f}")
    print(f"Precision: {precisions[idx]:.3f}")
    
    # Business impact analysis
    tn, fp, fn, tp = cm_gb.ravel()
    total_frauds = tp + fn
    total_legitimate = tn + fp
    
    print(f"\nBusiness Impact:")
    print(f"Total frauds in test set: {total_frauds}")
    print(f"Frauds caught: {tp} ({tp/total_frauds:.1%})")
    print(f"Frauds missed: {fn} ({fn/total_frauds:.1%})")
    print(f"Legitimate users flagged: {fp} ({fp/total_legitimate:.2%} of all legitimate users)")
    print(f"Legitimate users correctly cleared: {tn}")
    
else:
    print(f"No threshold found with recall ≥ {target_recall}")

In [ ]:

# Here I want to see if I settle for getting 60% of predicted fraud right; how many users will be actually hit wrongly and how many fraudsters we willl miss
# I think the final desicion could depend on company´s budget for chargebacks  - I would propose missing 10% of fraudulent users (0.885 recall) and being wrong with 50% of predictions; 
# since these are not that many users in relation to our total population, we are not losing that much
# In exchange if we compromised on blocking wrong people (0.1 precision) the win is not that big (?), only 5 additional fraudulent users  or so caught 


# Set your target precision
target_precision = 0.9

# Get predicted probabilities
y_probs = search_gb.predict_proba(X_test)[:, 1]

# Get precision-recall-thresholds
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)

# Find the first index where precision >= target (excluding the last point)
valid_idx = np.where(precisions[:-1] >= target_precision)[0]

if len(valid_idx) > 0:
    idx = valid_idx[0]
    threshold = thresholds[idx]
    
    y_pred = (y_probs >= threshold).astype(int)

    cm_gb = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_gb)
    disp.plot(cmap='Blues')

    print(f"Threshold for ≥ {target_precision:.2f} precision: {threshold:.3f}")
    print(f"Precision: {precisions[idx]:.3f}")
    print(f"Recall:    {recalls[idx]:.3f}")
else:
    print(f"No threshold found with precision ≥ {target_precision}")


# Debug the models

In [ ]:
# first lets check feature importance for all three models

lr_imp = np.abs(search_lr.best_estimator_['clf'].coef_).flatten()
rf_imp = search_rf.best_estimator_['clf'].feature_importances_
gb_imp = search_gb.best_estimator_['clf'].feature_importances_

# Helper to get sorted top N indices for a model
def top_features(imp, n=15):
    idx = np.argsort(imp)[-n:][::-1]
    return idx, imp[idx]

# Top 15 features for each model (sorted descending)
lr_idx, lr_vals = top_features(lr_imp)
rf_idx, rf_vals = top_features(rf_imp)
gb_idx, gb_vals = top_features(gb_imp)

# Plot
plt.figure(figsize=(18, 10))

plt.subplot(1, 3, 1)
plt.barh(range(15), lr_vals[::-1])
plt.yticks(range(15), lr_idx[::-1])
plt.title("Logistic Regression")

plt.subplot(1, 3, 2)
plt.barh(range(15), rf_vals[::-1])
plt.yticks(range(15), rf_idx[::-1])
plt.title("Random Forest")

plt.subplot(1, 3, 3)
plt.barh(range(15), gb_vals[::-1])
plt.yticks(range(15), gb_idx[::-1])
plt.title("Gradient Boosting")

plt.tight_layout()
plt.show()


# Note from heatmap:
## Features with highest positive corr to fraud are 2, 4, 11, 19
## Features with highest negative corr to fraud are 3, 9, 10, 12, 14, 17, 18, 20



# Notes on feature importance:
# Features 13 never appeares as correlated to Fraud in the heatmap but we are seeing it in the feature importance in all three models




In [ ]:
# Predict labels (0/1) for each model
y_pred_lr = search_lr.predict(X_test)
y_pred_rf = search_rf.predict(X_test)
y_pred_gb = search_gb.predict(X_test)

# Collect row index for comparison
row_ids = X_test.index if hasattr(X_test, "index") else np.arange(len(X_test))

# Identify false positives and false negatives
fp_lr = row_ids[(y_pred_lr == 1) & (y_test == 0)]
fn_lr = row_ids[(y_pred_lr == 0) & (y_test == 1)]

fp_rf = row_ids[(y_pred_rf == 1) & (y_test == 0)]
fn_rf = row_ids[(y_pred_rf == 0) & (y_test == 1)]

fp_gb = row_ids[(y_pred_gb == 1) & (y_test == 0)]
fn_gb = row_ids[(y_pred_gb == 0) & (y_test == 1)]


In [ ]:
# Overlaps in false positives
print("FP overlaps:")
print("LR & RF:", len(set(fp_lr).intersection(fp_rf)))
print("LR & GB:", len(set(fp_lr).intersection(fp_gb)))
print("RF & GB:", len(set(fp_rf).intersection(fp_gb)))

# Overlaps in false negatives
print("\nFN overlaps:")
print("LR & RF:", len(set(fn_lr).intersection(fn_rf)))
print("LR & GB:", len(set(fn_lr).intersection(fn_gb)))
print("RF & GB:", len(set(fn_rf).intersection(fn_gb)))


In [ ]:
# lets add predictions from all three models to the test set to analize the errors

X_test_df = pd.DataFrame(X_test).reset_index(drop=True)

y_test_series = pd.Series(y_test, name="true_label").reset_index(drop=True)
y_pred_lr_series = pd.Series(y_pred_lr, name="pred_lr")
y_pred_rf_series = pd.Series(y_pred_rf, name="pred_rf")
y_pred_gb_series = pd.Series(y_pred_gb, name="pred_gb")

df_all_preds = pd.concat([
    X_test_df,
    y_test_series,
    y_pred_lr_series,
    y_pred_rf_series,
    y_pred_gb_series
], axis=1)

df_all_preds.head()

### Deep Dive GB Results

In [ ]:

def classify_error(row):
    if row['true_label'] == 1 and row['pred_gb'] == 1:
        return "TP"
    elif row['true_label'] == 0 and row['pred_gb'] == 1:
        return "FP"
    elif row['true_label'] == 1 and row['pred_gb'] == 0:
        return "FN"
    else:
        return "TN"

df_all_preds["error_type_gb"] = df_all_preds.apply(classify_error, axis=1)


In [ ]:
# Define features to plot
features_to_plot = [13, 3, 11]
feature_names = ["Feature 13", "Feature 3", "Feature 11"]

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Loop through features
for i, (feature, name) in enumerate(zip(features_to_plot, feature_names)):
    sns.boxplot(data=df_all_preds, x="error_type_gb", y=feature, 
                order=["TP", "FP", "FN", "TN"], ax=axes[i])
    axes[i].set_title(f"{name} Distribution by Error Type")
    axes[i].set_ylabel(f"{name} Values")
    
    # Rotate x-axis labels if needed
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df_all_preds[[3, 13, 11, 'error_type_gb']], hue="error_type_gb", palette="Set2")
plt.suptitle("Pairplot of Key Features by Error Type", y=1.02)
plt.show()

In [ ]:
# Do false negatives have tight or different values?
# Do false positives overlap with true positives?
# Is there a range where mistakes consistently happen?


df_all_preds[df_all_preds["error_type_gb"]=="FN"].describe().transpose().round(2)

In [ ]:
df_all_preds[df_all_preds["error_type_gb"]=="FP"].describe().transpose().round(2)

### Review of overlapping errors

In [ ]:
# create dataframes for each error type that overlaps between different models

df_false_negatives = df_all_preds[(df_all_preds['true_label']==1) & 
             (df_all_preds['pred_lr']==0) &
            (df_all_preds['pred_rf']==0) & 
            (df_all_preds['pred_gb']==0)]

df_false_positives = df_all_preds[(df_all_preds['true_label']==0) & 
             (df_all_preds['pred_lr']==1) &
            (df_all_preds['pred_rf']==1) & 
            (df_all_preds['pred_gb']==1)]

df_true_positives = df_all_preds[(df_all_preds['true_label']==1) & 
             (df_all_preds['pred_lr']==1) &
            (df_all_preds['pred_rf']==1) & 
            (df_all_preds['pred_gb']==1)]


df_true_negatives = df_all_preds[(df_all_preds['true_label']==0) & 
             (df_all_preds['pred_lr']==0) &
            (df_all_preds['pred_rf']==0) & 
            (df_all_preds['pred_gb']==0)]

In [ ]:
df_false_positives.describe().transpose()

In [ ]:
# Among TP, certain features have huge variability

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

sns.boxplot(data=df_false_positives, ax=axes[0])
axes[0].set_title("False Positives")

sns.boxplot(data=df_true_positives, ax=axes[1])
axes[1].set_title("True Positives")


plt.xticks(rotation=90)
plt.show()

# Trying Ensemble Models

In [ ]:
voting_hard = VotingClassifier(
    estimators=[
        ('lr', search_lr.best_estimator_),
        ('rf', search_rf.best_estimator_),
        ('gb', search_gb.best_estimator_)
    ],
    voting='hard'
)

voting_hard.fit(X_train, y_train)
y_pred_hard = voting_hard.predict(X_test)


In [ ]:
voting_soft = VotingClassifier(
    estimators=[
        ('lr', search_lr.best_estimator_),
        ('rf', search_rf.best_estimator_),
        ('gb', search_gb.best_estimator_)
    ],
    voting='soft',
    weights=[1, 1, 2]  # optional: boost best model
)

voting_soft.fit(X_train, y_train)
y_prob_soft = voting_soft.predict_proba(X_test)[:, 1]
y_pred_soft = (y_prob_soft > 0.5).astype(int)  # set your own threshold


In [ ]:
stacked = StackingClassifier(
    estimators=[
        ('lr', search_lr.best_estimator_),
        ('rf', search_rf.best_estimator_),
        ('gb', search_gb.best_estimator_)
    ],
    final_estimator=LogisticRegression(),
    passthrough=True  # includes original features
)

stacked.fit(X_train, y_train)
y_pred_stacked = stacked.predict(X_test)


In [ ]:
plt.figure(figsize=(10, 10))

# Logistic Regression
prec, rec, _ = precision_recall_curve(y_test, search_lr.predict_proba(X_test)[:, 1])
ap_lr_test = average_precision_score(y_test, search_lr.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Random Forest
prec, rec, _ = precision_recall_curve(y_test, search_rf.predict_proba(X_test)[:, 1])
ap_rf_test = average_precision_score(y_test, search_rf.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Gradient Boosting
prec, rec, _ = precision_recall_curve(y_test, search_gb.predict_proba(X_test)[:, 1])
ap_gb_test = average_precision_score(y_test, search_gb.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Soft Voting
prec, rec, _ = precision_recall_curve(y_test, voting_soft.predict_proba(X_test)[:, 1])
ap_soft_test = average_precision_score(y_test, voting_soft.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

# Stacking
prec, rec, _ = precision_recall_curve(y_test, stacked.predict_proba(X_test)[:, 1])
ap_stack_test = average_precision_score(y_test, stacked.predict_proba(X_test)[:, 1])
plt.plot(rec, prec)

plt.xlabel("Recall - Of all the frauds in reality, how many did you catch?")
plt.ylabel("Precision - Of all the users you predicted as frauds, how many actually were frauds?")
plt.legend(["LR", "RF", "GB", "Soft Voting", "Stacking"])
plt.hlines(y=0.001727, xmin=0, xmax=1, colors='red', linestyles='dashed', label='Baseline (random)')


In [ ]:
print("LR AP: Test {:.3f}".format(ap_lr_test))
print("RF AP: Test {:.3f}".format(ap_rf_test))
print("GB AP: Test {:.3f}".format(ap_gb_test))
print("Soft Voting AP: Test {:.3f}".format(ap_soft_test))
print("Stacking AP: Test {:.3f}".format(ap_stack_test))
